In [1]:
from pathlib import Path
import glob
import sys
import os


os.environ['NOVA_HOME'] = '/home/projects/hornsteinlab/Collaboration/NOVA/'
os.environ['NOVA_DATA_HOME'] = f"{os.environ['NOVA_HOME']}/input"
print('NOVA_HOME is at', os.getenv('NOVA_HOME'))
sys.path.insert(1, os.getenv('NOVA_HOME'))


import numpy as np
import pandas as pd
import seaborn as sns
from markdown import markdown
import matplotlib.pyplot as plt

from cell_profiler.code.cp_effect_size_utils import CP_OUTPUTS_FOLDER, validate_cp_files, extract_path_parts, merge_on_group, collect_cp_results_by_cell_line, load_cp_results, get_features_per_image, get_aggregated_features_per_image, collect_all_features, assign_gene_group_Coyne_new, assign_plate_Coyne_new, assign_gene_group_Coyne_old, run_analysis_generate_report, print_mixedlm_conclusions, plot_cp_feature_grouped_by_gene
from manuscript.plot_config import PlotConfig

%load_ext autoreload    
%autoreload 2

NOVA_HOME is at /home/projects/hornsteinlab/Collaboration/NOVA/


In [2]:
CP_OUTPUTS_FOLDER = '/home/projects/hornsteinlab/Collaboration/NOVA/cell_profiler/outputs/cell_profiler_RUNS/Final_cp_analysis'

# dNLS 
ANALYSIS_TYPE = 'PB_profiling/dNLS_LSM14A'
output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'mixedlm_results.csv')                        
df_dNLS_LSM14A = pd.read_csv(output_dir)
print(f"df_dNLS_LSM14A: {df_dNLS_LSM14A.shape}")

ANALYSIS_TYPE = 'PB_profiling/dNLS_DCP1A'
output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'mixedlm_results.csv')                        
df_dNLS_DCP1A = pd.read_csv(output_dir)
print(f"df_dNLS_DCP1A: {df_dNLS_DCP1A.shape}")

# Coyne
ANALYSIS_TYPE = 'PB_profiling/coyne_old_DCP1A'   
output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'mixedlm_results.csv')
df_coyne_old_DCP1A = pd.read_csv(output_dir)
print(f"df_coyne_old_DCP1A: {df_coyne_old_DCP1A.shape}")

ANALYSIS_TYPE = 'PB_profiling/coyne_new_DCP1A'
output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'mixedlm_results.csv')
df_coyne_new_DCP1A = pd.read_csv(output_dir)
print(f"df_coyne_new_DCP1A: {df_coyne_new_DCP1A.shape}")


# V5 dNLS
ANALYSIS_TYPE = 'PB_profiling/WT_OE_DCP1A'
output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'TDP43_dNLS_Monocolonal', 'mixedlm_results.csv')
df_WT_OE_DCP1A_dNLS_Monocolonal = pd.read_csv(output_dir)
print(f"df_WT_OE_DCP1A_dNLS_Monocolonal: {df_WT_OE_DCP1A_dNLS_Monocolonal.shape}")

output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'TDP43_WT_Polycolonal', 'mixedlm_results.csv')
df_WT_OE_DCP1A_WT_Polycolonal = pd.read_csv(output_dir)
print(f"df_WT_OE_DCP1A_WT_Polycolonal: {df_WT_OE_DCP1A_WT_Polycolonal.shape}")

output_dir=os.path.join(CP_OUTPUTS_FOLDER, ANALYSIS_TYPE, 'mixed_effect_report', 'TDP43_dNLS_Polycolonal', 'mixedlm_results.csv')
df_WT_OE_DCP1A_dNLS_Polycolonal = pd.read_csv(output_dir)
print(f"df_WT_OE_DCP1A_dNLS_Polycolonal: {df_WT_OE_DCP1A_dNLS_Polycolonal.shape}")


df_dNLS_LSM14A: (82, 16)
df_dNLS_DCP1A: (82, 16)
df_coyne_old_DCP1A: (164, 16)
df_coyne_new_DCP1A: (164, 16)
df_WT_OE_DCP1A_dNLS_Monocolonal: (82, 16)
df_WT_OE_DCP1A_WT_Polycolonal: (82, 16)
df_WT_OE_DCP1A_dNLS_Polycolonal: (82, 16)


In [3]:
def get_sig(df):
    
    tmp = df[(df['significance']!='ns') & (df['comparison']!='Intercept')]
    return tmp.feature.unique()

sig_dNLS_DCP1A = get_sig(df=df_dNLS_DCP1A)
sig_dNLS_LSM14A = get_sig(df=df_dNLS_LSM14A)



In [4]:
print(len(sig_dNLS_DCP1A))
sig_dNLS_DCP1A
# 'num_pb', 'mean_AreaShape_Area', 'mean_AreaShape_Compactness','mean_AreaShape_Eccentricity', 'mean_Math_DCP1A_PB_over_cyto', 'mean_AreaShape_EquivalentDiameter','mean_AreaShape_FormFactor', 'mean_AreaShape_MeanRadius','mean_AreaShape_MedianRadius', 'mean_AreaShape_NormalizedMoment_1_1'

27


array(['num_pb', 'mean_AreaShape_Area', 'mean_AreaShape_Compactness',
       'mean_AreaShape_Eccentricity', 'mean_AreaShape_EquivalentDiameter',
       'mean_AreaShape_FormFactor', 'mean_AreaShape_MeanRadius',
       'mean_AreaShape_MedianRadius',
       'mean_AreaShape_NormalizedMoment_0_3',
       'mean_AreaShape_NormalizedMoment_1_1',
       'mean_AreaShape_NormalizedMoment_1_3',
       'mean_AreaShape_NormalizedMoment_2_1',
       'mean_AreaShape_NormalizedMoment_3_1',
       'mean_AreaShape_NormalizedMoment_3_3',
       'mean_Math_DCP1A_PB_over_cyto',
       'mean_Math_Texture_Contrast_DCP1A_pb_only_15',
       'mean_Math_Texture_Contrast_DCP1A_pb_only_3',
       'mean_Math_Texture_Contrast_DCP1A_pb_only_5',
       'mean_Math_Texture_Contrast_DCP1A_pb_only_9',
       'mean_Math_Texture_Entropy_DCP1A_pb_only_15',
       'mean_Math_Texture_Entropy_DCP1A_pb_only_3',
       'mean_Math_Texture_Entropy_DCP1A_pb_only_5',
       'mean_Math_Texture_Entropy_DCP1A_pb_only_9',
       'mean_Ma

In [5]:
print(len(sig_dNLS_LSM14A))
sig_dNLS_LSM14A
# ['mean_AreaShape_Compactness', 'mean_AreaShape_Eccentricity', 'mean_Math_LSM14A_PB_over_cyto',, 'mean_AreaShape_MeanRadius', 'mean_AreaShape_MedianRadius', 'mean_AreaShape_NormalizedMoment_1_1', 'mean_Math_Texture_Contrast_LSM14A_pb_only_3','mean_Math_Texture_Entropy_LSM14A_pb_only_3', 'mean_Math_Texture_Homogeneity_LSM14A_pb_only_3'
       

24


array(['mean_AreaShape_Compactness', 'mean_AreaShape_Eccentricity',
       'mean_AreaShape_HuMoment_0', 'mean_AreaShape_HuMoment_1',
       'mean_AreaShape_HuMoment_2', 'mean_AreaShape_HuMoment_3',
       'mean_AreaShape_HuMoment_4', 'mean_AreaShape_HuMoment_5',
       'mean_AreaShape_MeanRadius', 'mean_AreaShape_MedianRadius',
       'mean_AreaShape_NormalizedMoment_0_2',
       'mean_AreaShape_NormalizedMoment_1_1',
       'mean_AreaShape_NormalizedMoment_2_0',
       'mean_AreaShape_NormalizedMoment_2_2',
       'mean_Math_LSM14A_PB_over_cyto',
       'mean_Math_Texture_Contrast_LSM14A_pb_only_15',
       'mean_Math_Texture_Contrast_LSM14A_pb_only_3',
       'mean_Math_Texture_Contrast_LSM14A_pb_only_5',
       'mean_Math_Texture_Contrast_LSM14A_pb_only_9',
       'mean_Math_Texture_Entropy_LSM14A_pb_only_3',
       'mean_Math_Texture_Entropy_LSM14A_pb_only_5',
       'mean_Math_Texture_Entropy_LSM14A_pb_only_9',
       'mean_Math_Texture_Homogeneity_LSM14A_pb_only_3',
       'mean_

In [53]:

np.intersect1d(sig_dNLS_DCP1A, sig_dNLS_LSM14A)



27


array(['mean_AreaShape_Compactness', 'mean_AreaShape_Eccentricity',
       'mean_AreaShape_MeanRadius', 'mean_AreaShape_MedianRadius',
       'mean_AreaShape_NormalizedMoment_1_1'], dtype=object)

In [35]:
def get_sig_patients(df):
    
    tmp = df[(df['significance']!='ns') & (df['comparison']=='C9')]
    return tmp.feature.unique()

sig_coyne_new_DCP1A = get_sig_patients(df=df_coyne_new_DCP1A)
print(sig_coyne_new_DCP1A)
df_coyne_new_DCP1A[df_coyne_new_DCP1A.feature.isin(sig_coyne_new_DCP1A)]

# 'num_pb', 'mean_AreaShape_Eccentricity', 'mean_AreaShape_EquivalentDiameter', 'mean_AreaShape_FormFactor', 'mean_AreaShape_MeanRadius', mean_Math_Texture_Contrast_DCP1A_pb_only_3', 'mean_Math_Texture_Entropy_DCP1A_pb_only_3'


['num_pb' 'mean_AreaShape_Eccentricity'
 'mean_AreaShape_EquivalentDiameter' 'mean_AreaShape_FormFactor'
 'mean_AreaShape_HuMoment_0' 'mean_AreaShape_HuMoment_1'
 'mean_AreaShape_HuMoment_2' 'mean_AreaShape_HuMoment_3'
 'mean_AreaShape_HuMoment_5' 'mean_AreaShape_MeanRadius'
 'mean_AreaShape_NormalizedMoment_0_2'
 'mean_AreaShape_NormalizedMoment_2_0'
 'mean_AreaShape_NormalizedMoment_2_2'
 'mean_Math_Texture_Contrast_DCP1A_pb_only_3'
 'mean_Math_Texture_Entropy_DCP1A_pb_only_3']


,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
0,num_pb,Intercept,6.573427e+00,3.980652e-60,6.195635e+00,6.951219e+00,NaN,3.508089e+00,***,fallback_ols_fixed_batch,True,502.510549,535.960450,-239.255274,0.676177,2.967395e-59
1,num_pb,C9,1.720105e+00,1.512828e-03,6.727476e-01,2.767462e+00,NaN,3.508089e+00,**,fallback_ols_fixed_batch,True,502.510549,535.960450,-239.255274,0.676177,3.140554e-03
2,num_pb,SALSNegative,2.576639e+00,2.657047e-16,2.048357e+00,3.104920e+00,NaN,3.508089e+00,***,fallback_ols_fixed_batch,True,502.510549,535.960450,-239.255274,0.676177,8.892975e-16
3,num_pb,SALSPositive,-1.178845e+00,2.329454e-05,-1.707127e+00,-6.505629e-01,NaN,3.508089e+00,***,fallback_ols_fixed_batch,True,502.510549,535.960450,-239.255274,0.676177,5.877392e-05
12,mean_AreaShape_Eccentricity,Intercept,4.070985e-01,1.218283e-133,4.024151e-01,4.117818e-01,NaN,5.391059e-04,***,fallback_ols_fixed_batch,True,-551.169842,-517.719941,287.584921,0.349368,1.816349e-132
13,mean_AreaShape_Eccentricity,C9,8.237663e-02,6.945545e-23,6.939299e-02,9.536027e-02,NaN,5.391059e-04,***,fallback_ols_fixed_batch,True,-551.169842,-517.719941,287.584921,0.349368,2.712070e-22
14,mean_AreaShape_Eccentricity,SALSNegative,7.552902e-02,3.483932e-43,6.898013e-02,8.207790e-02,NaN,5.391059e-04,***,fallback_ols_fixed_batch,True,-551.169842,-517.719941,287.584921,0.349368,1.843112e-42
15,mean_AreaShape_Eccentricity,SALSPositive,5.633450e-02,2.085868e-32,4.978561e-02,6.288339e-02,NaN,5.391059e-04,***,fallback_ols_fixed_batch,True,-551.169842,-517.719941,287.584921,0.349368,8.771343e-32
16,mean_AreaShape_EquivalentDiameter,Intercept,3.434855e+00,2.649308e-138,3.399091e+00,3.470619e+00,NaN,3.143873e-02,***,fallback_ols_fixed_batch,True,-63.263761,-29.813860,43.631880,0.350877,4.344866e-137
17,mean_AreaShape_EquivalentDiameter,C9,5.520917e-01,2.034566e-19,4.529419e-01,6.512415e-01,NaN,3.143873e-02,***,fallback_ols_fixed_batch,True,-63.263761,-29.813860,43.631880,0.350877,7.583382e-19


In [55]:
df_coyne_new_DCP1A.loc[df_coyne_new_DCP1A.feature=='mean_Math_Texture_Contrast_DCP1A_pb_only_3', ['comparison','effect_size', 'pval']]

,comparison,effect_size,pval
120,Intercept,54.150411,1.316119e-54
121,C9,12.520618,1.297086e-02
122,SALSNegative,-4.925814,5.132955e-02
123,SALSPositive,4.257158,9.142653e-02


In [41]:
df_coyne_new_DCP1A.loc[df_coyne_new_DCP1A.feature=='mean_AreaShape_Eccentricity', ['comparison','effect_size', 'pval']]

,comparison,effect_size,pval
12,Intercept,0.407098,1.218283e-133
13,C9,0.082377,6.945545e-23
14,SALSNegative,0.075529,3.483932e-43
15,SALSPositive,0.056334,2.085868e-32


In [60]:
df_dNLS_DCP1A[df_dNLS_DCP1A.feature=='num_pb']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
0,num_pb,Intercept,69.443140,0.00000,66.491310,72.394970,NaN,972.311472,***,fallback_ols_fixed_batch,True,38594.399474,38632.120114,-19291.199737,0.268522,0.000000
1,num_pb,dNLS_DOX,-4.420578,0.00001,-6.379988,-2.461169,NaN,972.311472,***,fallback_ols_fixed_batch,True,38594.399474,38632.120114,-19291.199737,0.268522,0.000014


In [56]:

df_dNLS_DCP1A[df_dNLS_DCP1A.feature=='mean_Math_Texture_Contrast_DCP1A_pb_only_3']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
60,mean_Math_Texture_Contrast_DCP1A_pb_only_3,Intercept,219.298175,1.283325e-49,190.667462,247.928888,NaN,91471.767652,***,fallback_ols_fixed_batch,True,56639.058123,56676.778762,-28313.529061,0.446479,2.505540e-49
61,mean_Math_Texture_Contrast_DCP1A_pb_only_3,dNLS_DOX,-272.728877,5.927754e-159,-291.733796,-253.723958,NaN,91471.767652,***,fallback_ols_fixed_batch,True,56639.058123,56676.778762,-28313.529061,0.446479,1.676124e-158


In [32]:
df_dNLS_LSM14A[df_dNLS_LSM14A.feature=='mean_AreaShape_Eccentricity']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
6,mean_AreaShape_Eccentricity,Intercept,0.620299,0.000000e+00,0.614104,0.626494,NaN,0.007109,***,fallback_ols_fixed_batch,True,-10197.419033,-10158.512253,5104.709517,0.283863,NaN
7,mean_AreaShape_Eccentricity,dNLS_DOX,0.019187,3.343378e-15,0.014428,0.023947,NaN,0.007109,***,fallback_ols_fixed_batch,True,-10197.419033,-10158.512253,5104.709517,0.283863,NaN


In [57]:

df_WT_OE_DCP1A_WT_Polycolonal[df_WT_OE_DCP1A_WT_Polycolonal.feature=='mean_Math_Texture_Contrast_DCP1A_pb_only_3']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
60,mean_Math_Texture_Contrast_DCP1A_pb_only_3,Intercept,53.580152,5.292901e-32,44.753999,62.406304,NaN,12210.475122,***,fallback_ols_fixed_batch,True,39650.535754,39674.865363,-19821.267877,0.129799,7.614349e-32
61,mean_Math_Texture_Contrast_DCP1A_pb_only_3,TDP43_WT_Polycolonal_DOX,81.429765,7.065642e-85,73.492014,89.367515,NaN,12210.475122,***,fallback_ols_fixed_batch,True,39650.535754,39674.865363,-19821.267877,0.129799,1.485596e-84


In [58]:
df_WT_OE_DCP1A_dNLS_Polycolonal[df_WT_OE_DCP1A_dNLS_Polycolonal.feature=='mean_Math_Texture_Contrast_DCP1A_pb_only_3']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
60,mean_Math_Texture_Contrast_DCP1A_pb_only_3,Intercept,43.199269,1.858257e-34,36.430269,49.968269,NaN,3263.838188,***,fallback_ols_fixed_batch,True,18713.649305,18735.430976,-9352.824653,0.021804,3.463115e-34
61,mean_Math_Texture_Contrast_DCP1A_pb_only_3,TDP43_dNLS_Polycolonal_DOX,10.162743,1.872060e-03,3.763127,16.562359,NaN,3263.838188,**,fallback_ols_fixed_batch,True,18713.649305,18735.430976,-9352.824653,0.021804,2.257484e-03


In [59]:
df_WT_OE_DCP1A_dNLS_Monocolonal[df_WT_OE_DCP1A_dNLS_Monocolonal.feature=='mean_Math_Texture_Contrast_DCP1A_pb_only_3']

,feature,comparison,effect_size,pval,ci_lower,ci_upper,group_var,residual_var,significance,fit_status,used_fixed_model,aic,bic,loglik,r_squared,fdr_pval_global
60,mean_Math_Texture_Contrast_DCP1A_pb_only_3,Intercept,256.586858,0.000000e+00,251.440813,261.732904,NaN,7022.799408,***,fallback_ols_fixed_batch,True,46502.499964,46527.652091,-23247.249982,0.203714,0.000000e+00
61,mean_Math_Texture_Contrast_DCP1A_pb_only_3,TDP43_dNLS_Monocolonal_DOX,-82.998631,2.862525e-191,-88.215069,-77.782192,NaN,7022.799408,***,fallback_ols_fixed_batch,True,46502.499964,46527.652091,-23247.249982,0.203714,9.389082e-191
